In [20]:
#!pip install --quiet langchain-core==0.3.59 langgraph==0.5.3 langchain-openai==0.3.16 langchain-experimental==0.3.4 pydantic==2.11.9
!pip install youdotcom


  Obtaining dependency information for youdotcom from https://files.pythonhosted.org/packages/db/c2/e4bb6bf71b41a2f6d83949ac033f7dbc8f801e068578af2961610608c9de/youdotcom-1.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore>=1.0.9 from https://files.pythonhosted.org/packages/7e/f5/f66802a942d491edb555dd61e3a9961140fd64c90bce1eafd741609d334d/httpcore-1.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for h11>=0.16 from https://files.pythonhosted.org/packages/04/4b/29cac41a4d98d144bf5f6d33995617b185d14b22401f75ca86f384e87ff1/h11-0.16.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.7
    Uninst

In [21]:
from typing import Annotated
from langchain_core.tools import tool
import pandas as pd
import os
from langchain_experimental.utilities import PythonREPL
import requests

repl = PythonREPL()

from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# Tools

In [7]:
@tool
def fpl_data_tool(
    query: Annotated[str, "The FPL data to retrieve. Options are 'players', 'teams', 'fixtures', 'events'."],
):
    """Use this to retrieve FPL data from the official FPL API."""
    base_url = "https://fantasy.premierleague.com/api/"
    endpoints = {
        "players": "bootstrap-static/",
        "teams": "teams/",
        "fixtures": "fixtures/",
        "events": "events/",
    }
    if query not in endpoints:
        return f"Invalid query. Please choose from {list(endpoints.keys())}."
    try:
        response = requests.get(base_url + endpoints[query])
        response.raise_for_status()
        data = response.json()
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Successfully executed:\nFPL {query} data retrieved with {len(data)} records."

@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user. The chart should be displayed using `plt.show()`."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Successfully executed the Python REPL tool.\n\nPython code executed:\n\`\`\`python\n{code}\n\`\`\`\n\nCode output:\n\`\`\`\n{result}\`\`\`"

In [8]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tools = [fpl_data_tool, python_repl_tool]

llm = ChatOpenAI(model="gpt-4o-mini")

# Tell the LLM which tools it can call
llm_with_tools = llm.bind_tools(tools)

def llm_node(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable